# Réseaux neuronaux convolutifs

Au dernier cours, nous avons vu comment construire un réseau neuronal simple et pleinement connecté ainsi que comment l'appliquer à la classification d'images.
Nous avons aussi couvert comment entraîner ce réseau et l'appliquer à des données test, ou encore à des données provenant de sources externes.

Or, le réseau pleinement connecté que nous utilisions n'était pas nécessairement le plus adapté  à la classification d'images. Pour ce genre de tâche un réseau neuronal convolutif est généralement plus approprié.
Nous verrons aujourd'hui comment implémenter un tel réseau avec PyTorch.

## Données

Nous allons commencer par appliquer notre modèle aux mêmes données qu'au dernier cours, soit l'ensemble FashionMNIST.

In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor

device = "cuda" if torch.cuda.is_available() else "cpu"

training_data = datasets.FashionMNIST(
    root="../cours03-images-mlp-pytorch/data",  # On réutilise le même dossier pour ne pas télécharger en double
    train=True,
    download=True,
    transform=ToTensor(),  # Transformation de tableau PIL vers tenseur
)
test_data = datasets.FashionMNIST(
    root="../cours03-images-mlp-pytorch/data",
    train=False,
    download=True,
    transform=ToTensor(),
)

labels_map = {
    0: "T-Shirt",
    1: "Trouser",
    2: "Pullover",
    3: "Dress",
    4: "Coat",
    5: "Sandal",
    6: "Shirt",
    7: "Sneaker",
    8: "Bag",
    9: "Ankle Boot",
}
inv_labels_map = dict(zip(labels_map.values(), labels_map.keys()))

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt

def plot_grid(data):
    """
    Fonction pour tirer 9 images au hasard d'un Dataset PyTorch et les fafficher
    """
    figure = plt.figure(figsize=(8, 8))
    cols, rows = 3, 3
    for i in range(1, cols * rows + 1):
        # On tire une image au hasard et on l'affiche
        sample_idx = torch.randint(len(data), size=(1,)).item()
        img, label = data[sample_idx]
        figure.add_subplot(rows, cols, i)
        plt.title(labels_map[label])
        plt.axis("off")
        plt.imshow(img.permute(1, 2, 0), cmap="binary")
    return figure
    
plot_grid(training_data)
plt.show()

## Définition d'un CNN

### Couche de convolution

Commençons par nous familiariser avec la couche de convolution 2D de PyTorch, soit [nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html).

**Exercices:**

1. Le premier paramètre de `Conv2d` est le nombre de canaux en entrée. Combien de canaux d'entrée une couche de convolution utilisée sur les données Fashion MNST?
2. Créez un modèle simple en assignant directement `model` à une couche de convolution, par exemple `model = nn.Conv2d(*args)` où `*args` est remplacé par les arguments requis. La couche de convolution devrait avoir le nombre de canaux d'entrée adéquat pour les données Fahsion MNIST et 8 canaux de sortie. Utilisez un noyaux de 5 pixels pour la convolution.
3. Affichez les paramètres de ce modèle. Quel est le format (`shape`) des poids ?
4. Passez une des images d'entraînement dans le modèle. Quelles sont les dimensions de sortie?

In [ ]:
from torch import nn

model = None # TODO: Remplacez None par une couche de convolution

In [ ]:
# TODO: Utilisez .named_parameters() ou .paramters() pour accéder aux poids et afficher leur format

In [ ]:
# TODO: Accès à une image et utilisation du modèle

### Calcul des dimensions de sortie

Pour une seule couche de convolution, il est facile de vérifier les dimensions de sortie. Pour se faciliter la vie, on peut également définir une fonction qui fait ce calcul. Pour un padding de type `same`, c'est assez facile: les dimensions de sortie sont égales à celles d'entrée. Or, pour un padding `valid`, ce ne sera pas le cas. De manière générale, les dimensions de sortie d'une convolution sont données par l'équation ci-dessous.

Pour une image de hauteur (et largeur) $H_in$, la hauteur (et largeur) de la couche de sortie est donnée par:

$$
H_{out} = \left\lfloor\frac{H_{in}  + 2 \times \text{padding} - \text{dilation}
                        \times (\text{kernel\_size} - 1) - 1}{\text{stride}} + 1\right\rfloor
$$

Pour nous, dans la plupart des cas:
- `padding=0` car on utilise une convolution "valid" par défaut
- `stride=1` car on glisse le noyau de convolution pixel par pixel
- `dilation=1`: on ne dilate pas le noyau

On peut voir intuitivement que:

- Un plus padding plus grand augmente la dimension de sortie
- Un `stride` (un pas) plus grand diminue les dimensions (on répète la convolution à moinds d'emplacements sur l'image)
- Un noyau plus grand résulte en une dimension de sortie plus petite: plus les pixels du noyau couvrent une grande partie de l'image, moins on a de jeu pour glisser la convolution. C'est un peu la même idée avec la dilatation.

**Exercice: Implémentez une fonction qui calcule cette équation et vérifiez son résultat avec la convolution ci-dessus.**

In [ ]:
def get_output_width(input_width, kernel_size, stride=1, padding=0, dilation=1):
    """
    - input_wdith: taille de l'entrée
    - kernel_size: taille du noyau
    - stride: pas effectué par le noyau convolution
    - padding: nombre de pixels de padding (0 pour valid, varie pour same)
    - dilation: Espacement entre les pixels du noyau
    # Ref: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d
    **Attention: stride=kernel_size pour max pooling par défaut**
    """
    # TODO: Implémentation et test

### Padding différent

**Exercice: Utilisez un modèle alternatif avec un padding 'same'. Le format de la couche de sortie est-il de la manière attendue?**

In [ ]:
# TODO: Convolution et test

### Mise en commun (_pooling_)

PyTorch implémente aussi des couches de mise en commun: [nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html).

**Exercice: Implémentez une couche de mise en commun de dimension 5 et utilisez-la sur une image d'entraînement. Quelles sont les dimensions de sortie? Comment diffèrent-elles d'une convolution? Est-ce cohérent avec l'équation vue plus haut? Affichez sur deux graphiques côte à côte l'image initiale et le résultat du pooling.**

<details>
    <summary>Explication concernant la taille de sortie</summary>
    Par défaut, pour `MaxPool2d`, `stride=kernel_size`.
</details>

In [ ]:
# TODO: Pooling, utilisation et visualisation

### Définition d'un CNN complet

Pour définir un CNN, il suffit de combiner une ou plusieurs couches de convolution comme celle définie ci-dessus dans un réseau neuronal.

**Exercice: Définissez un CNN avec les caractéristiques ci-dessous. Essayez d'abord de le faire sans retourner voir l'exemple du dernier cours, mais n'hésitez pas à vous y référer au besoin.**

- Une première couche de convolution avec 6 canaux de sortie et un noyaux de largeur 5
- Une deuxième couche de convolution avec 16 canaux de sortie et un noyaux de largeur 5
- Trois couches pleinement connectées avec 120, 84 et 10 neurones
- Une fonction d'activation ReLU pour toutes les couches sauf la dernière
- Un pooling "max" 2x2 après l'activation des couches de convolution.
- Dans l'exemple du dernier cours, nous avons utilisé `nn.Flatten` pour applatir les images. Nous en aurons besoin ici également. À quel endroit dans le réseau doit on applatir les données?

In [ ]:
# TODO: Réseau CNN

**Exercice: créez une instance de votre modèle. Quelle est la dimension d'entrée de la première couche pleinement connectée?**

In [ ]:
model = ConvNet().to(device)
# TODO: Accès à la dimension de la couche linéaire

### Test rapide du modèle

**Exercice: Assurez-vous que votre modèle fonctionne sur une image tirée de `training_data` et un sous-ensemble tiré de `train_dataloader`.**
Pour l'image seule, n'oubliez pas d'ajouter la dimension de "batch" avec `img.unsqueeze()`.

In [ ]:
# TODO: Test du modèle

## Entraînement

**Exercie: Entraînez le réseau avec une fonction objectif _cross-entropy_.**

Essayez de vous référer à l'exemple du dernier cours le moins possible, mais consultez le au besoin.

Rappel des étapes:

- Définition de la fonction objectif et de l'optimiseur
- Définition d'une boucle d'entraînement et d'une boucle de test
- Itération sur les époques (commencez par 10)

In [ ]:
# TODO: Fonction objectif et optimiseur

In [ ]:
# TODO: Boucle d'entraînement

In [ ]:
# TODO: Boucle de test

In [ ]:
# TODO: Entraînement par époque

**Exercice: Affichez l'évolution de la fonciton objectif pour les données d'entraînement et de test.**

In [ ]:
# TODO: Graphique

Remarquez que la performance après 10 époques est moins bonne que pour le réseau pleinement connecté du dernier cours! Ceci s'explique en partie le fait que notre réseau est très simple et en partie par le fait que nous avons coupé l'entraînement relativement tôt.

## Inspection du modèle

Comme au dernier cours, on pourrait utiliser le modèle entraîner pour prédire différents exemples. Le code serait pratiquement le même.

Par contre, comme nous avons un CNN ici, on peut essayer d'interpréter les poids dans les différentes convolutions.

**Exercice: Accédez à la première couche de convolution. Ensuite, accédez à ses poids et affichez le poids pour les différents canaux.**

<details>
    <summary>Indice</summary>
    Il faudra d'abord accéder à `conv_stack` et `conv_stack` est comme une liste, donc on peut l'indexer.
</details>

In [ ]:
# TODO: Accès à la couche de convolution et affichage

Assez difficile d'interpréter quoi que ce soit ici!

**Exercice: Affichez maintenant les sorties de la première couche pour une image tirée au hasard des données d'entraînement**.

In [ ]:
# TODO: Utilisation de la couche de convolution et affichage

## Exercices additionnels

- Modifiez le réseau ci-dessus en ajoutant des canaux, des couches et/ou en modifiant la taille des noyaux de convolutions et explorez l'effet sur le résultat.
- Modifiez le réseau ci-dessus afin qu'il accepte des images RGB avec 3 canaux et testez le sur les données CIFAR-10 (` torchvision.datasets.CIFAR10`)
- Utilisez un "dropout" dans le modèle
- Utilisez une "Batch normalization"